In [6]:
import os
import shutil
import platform
import datetime

import matplotlib.pyplot as plt
import seaborn as sns

import statsmodels.api as sm
import pandas as pd
import numpy as np
import scipy as sp
from typing import Union

from IPython.display import display, Markdown
from pandas.plotting import scatter_matrix

import quandl
with open('api_key.txt', 'r') as file:
    API_KEY = file.read().strip()
    
HOME_DIR = r'/Users/alexhuang/Documents/Academic/FINM 33150'
# HOME_DIR = r'C:\Users\Alex\Desktop\Academic\UChicago\FINM 33150'

In [7]:
class DataFetching:
    def grab_quandl_table(
        table_path,
        avoid_download=False,
        replace_existing=False,
        date_override=None,
        allow_old_file=False,
        **kwargs,
    ):
        root_data_dir = os.path.join(HOME_DIR, "quandl_data_table_downloads")
        data_symlink = os.path.join(root_data_dir, f"{table_path}_latest.zip")
        if avoid_download and os.path.exists(data_symlink):
            print(f"Skipping any possible download of {table_path}")
            return data_symlink
        
        table_dir = os.path.dirname(data_symlink)
        if not os.path.isdir(table_dir):
            print(f'Creating new data dir {table_dir}')
            os.makedirs(table_dir)

        if date_override is None:
            my_date = datetime.datetime.now().strftime("%Y%m%d")
        else:
            my_date = date_override
        data_file = os.path.join(root_data_dir, f"{table_path}_{my_date}.zip")

        if os.path.exists(data_file):
            file_size = os.stat(data_file).st_size
            if replace_existing or not file_size > 0:
                print(f"Removing old file {data_file} size {file_size}")
            else:
                print(
                    f"Data file {data_file} size {file_size} exists already, no need to download"
                )
                return data_file

        dl = quandl.export_table(
            table_path, filename=data_file, api_key=API_KEY, **kwargs
        )
        file_size = os.stat(data_file).st_size
        if os.path.exists(data_file) and file_size > 0:
            print(f"Download finished: {file_size} bytes")
            if not date_override:
                if os.path.exists(data_symlink):
                    print(f"Removing old symlink")
                    os.unlink(data_symlink)
                print(f"Creating symlink: {data_file} -> {data_symlink}")
                current_os = platform.system()
                if current_os == 'Windows':
                    shutil.copy(
                        data_file, data_symlink
                    )
                else:
                    os.symlink(
                        data_file, data_symlink,
                    )
        else:
            print(f"Data file {data_file} failed download")
            return
        return data_symlink if (date_override is None or allow_old_file) else "NoFileAvailable"



    def fetch_quandl_table(table_path, avoid_download=True, **kwargs) -> pd.DataFrame:
        return pd.read_csv(
            DataFetching.grab_quandl_table(table_path, avoid_download=avoid_download, **kwargs)
        )




In [9]:
ZACKS_FC = DataFetching.fetch_quandl_table('ZACKS/FC', avoid_download=False)

Data file /Users/alexhuang/Documents/Academic/FINM 33150/quandl_data_table_downloads/ZACKS/FC_20240131.zip size 187714472 exists already, no need to download


/var/folders/tl/2mnl94rs3r36jtm93n995fhr0000gn/T/ipykernel_67443/622141614.py:65: DtypeWarning: Columns (4,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,44,45,46,47,49,50,51,52,53,54,55,56,57,58,59) have mixed types. Specify dtype option on import or set low_memory=False.
  return pd.read_csv(


In [10]:
PRICES = DataFetching.fetch_quandl_table('QUOTEMEDIA/PRICES', ticker=['LLY'], avoid_download=False)

Download finished: 519119 bytes
Removing old symlink
Creating symlink: /Users/alexhuang/Documents/Academic/FINM 33150/quandl_data_table_downloads/QUOTEMEDIA/PRICES_20240131.zip -> /Users/alexhuang/Documents/Academic/FINM 33150/quandl_data_table_downloads/QUOTEMEDIA/PRICES_latest.zip


In [18]:
def preprocess_eps(zacks_fc_table: pd.DataFrame, fill_neg_eps: float=0.001, filter_per_type: str='Q'):
        # Validation
        validation_columns = ['m_ticker', 'per_end_date', 'per_type', 'filing_date', 'eps_diluted_net', 'basic_net_eps']
        for col in validation_columns:
                if col not in zacks_fc_table.columns:
                        raise KeyError(f"{col} not found in dataframe. {col} is required for EPS preprocessing.")

        # Cleaning table
        eps_df = (zacks_fc_table[zacks_fc_table['per_type'] == filter_per_type][validation_columns]
                .dropna(subset=['m_ticker'])
                .dropna(how='all', subset=['eps_diluted_net', 'basic_net_eps'])
                .sort_values(by=['m_ticker', 'per_end_date'])
                )
        
        # Cleaning and Formating Columns
        eps_df['m_ticker'] = eps_df['m_ticker'].str.strip()
        eps_df['per_end_date'] = pd.to_datetime(eps_df['per_end_date'], format='%Y-%m-%d')
        eps_df['filing_date'] = pd.to_datetime(eps_df['filing_date'], format='%Y-%m-%d')
        eps_df['tradable_date'] = eps_df['filing_date'] + pd.Timedelta(days=1)

        eps_df['processed_eps'] = eps_df['eps_diluted_net']
        eps_df['processed_eps'] = eps_df['processed_eps'].fillna(eps_df['basic_net_eps'])
        eps_df['processed_eps'] = eps_df['processed_eps'].apply(lambda x: max(fill_neg_eps, x))
        
        eps_df = eps_df.set_index(['m_ticker', 'tradable_date']).sort_index(ascending=[True, True])
        return eps_df

eps_df = preprocess_eps(
    ### PUT RAW ZACKS/FC TABLE HERE
    ZACKS_FC
)
display(Markdown("""
### The table below only has `tradable_date` on the dates that a filing had occured. We need to expand this and ffill with next function.
***
"""))
eps_df


### The table below only has `tradable_date` on the dates that a filing had occured. We need to expand this and ffill with next function.
***


per_end_date per_type filing_date  eps_diluted_net  \
m_ticker tradable_date                                                      
#AAO     2006-05-11      2006-03-31        Q  2006-05-10           0.0395   
         2006-08-10      2006-06-30        Q  2006-08-09           0.0356   
         2006-11-09      2006-09-30        Q  2006-11-08           0.0553   
         2007-03-15      2006-12-31        Q  2007-03-14           0.0474   
         2007-05-10      2007-03-31        Q  2007-05-09           0.0652   
...                             ...      ...         ...              ...   
ZZ       2012-01-19      2011-11-30        Q  2012-01-18          -0.1300   
         2012-03-28      2012-02-29        Q  2012-03-27           0.0100   
         2012-06-27      2012-05-31        Q  2012-06-26           0.0200   
         2012-09-29      2012-08-31        Q  2012-09-28           0.0000   
         2013-02-05      2012-11-30        Q  2013-02-04          -0.0299   

                        basic_net_eps  processed_eps  
m_ticker tradable_date                                
#AAO     2006-05-11            0.0395         0.0395  
         2006-08-10            0.0369         0.0356  
         2006-11-09            0.0579         0.0553  
         2007-03-15            0.0487         0.0474  
         2007-05-10            0.0672         0.0652  
...                               ...            ...  
ZZ       2012-01-19               NaN         0.0010  
         2012-03-28               NaN         0.0100  
         2012-06-27               NaN         0.0200  
         2012-09-29               NaN         0.0010  
         2013-02-05               NaN         0.0010  

[448851 rows x 6 columns]

In [14]:
def extract_expand_ffill(time_series: pd.Series, freq: str='D') -> pd.Series:
    """
    Expand and forward-fill a time series to fill missing values at regular intervals.

    Parameters:
    - time_series (pd.Series): The input time series with a datetime index.
    - freq (str, optional): The frequency string representing the desired frequency of the output time series.
                            Default is 'D' (daily).

    Returns:
    - pd.Series: The expanded and forward-filled time series.

    Example:
    >>> import pandas as pd
    >>> ts = pd.Series([1, 2, 3], index=pd.to_datetime(['2022-01-01', '2022-01-03', '2022-01-05']))
    >>> result = extract_expand_ffill(ts, freq='D')
    >>> print(result)
    2022-01-01    1.0
    2022-01-02    1.0  # Forward-filled from the previous day
    2022-01-03    2.0
    2022-01-04    2.0  # Forward-filled from the previous day
    2022-01-05    3.0
    Freq: D, dtype: float64
    """
    # Reindex the time series to cover the entire date range with the specified frequency and forward-fill missing values
    # time_series = time_series.sort_index()
    if isinstance(time_series, pd.Series):
        time_series = time_series.reindex(pd.date_range(start=time_series.index.min(), end=time_series.index.max(), freq=freq)).ffill()
    elif isinstance(time_series, pd.DataFrame):
        time_series = time_series.reindex(pd.date_range(start=time_series.index.min(), end=time_series.index.max(), freq=freq)).ffill()
    else:
        raise TypeError(f"time_series is not a pd.Series or pd.Dataframe object.")
    return time_series

In [22]:
PRICES['date'] = pd.to_datetime(PRICES['date'])
lly_prices = extract_expand_ffill(PRICES.set_index('date').sort_index())
display(Markdown("""
### The table below is foward filled so that we have prices on the weekends. This makes indexing easier.
***
"""))
display(lly_prices)


### The table below is foward filled so that we have prices on the weekends. This makes indexing easier.
***


,ticker,open,high,low,close,volume,dividend,split,adj_open,adj_high,adj_low,adj_close,adj_volume
1972-06-01,LLY,65.03,65.12,64.38,64.75,28500.0,0.0,1.0,1.482132,1.484184,1.467318,1.475751,456000.0
1972-06-02,LLY,64.88,65.50,64.88,65.50,40000.0,0.0,1.0,1.478714,1.492844,1.478714,1.492844,640000.0
1972-06-03,LLY,64.88,65.50,64.88,65.50,40000.0,0.0,1.0,1.478714,1.492844,1.478714,1.492844,640000.0
1972-06-04,LLY,64.88,65.50,64.88,65.50,40000.0,0.0,1.0,1.478714,1.492844,1.478714,1.492844,640000.0
1972-06-05,LLY,65.38,65.38,64.38,64.62,29000.0,0.0,1.0,1.490109,1.490109,1.467318,1.472788,464000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-01-26,LLY,637.35,639.60,630.50,639.25,2385532.0,0.0,1.0,637.350000,639.600000,630.500000,639.250000,2385532.0
2024-01-27,LLY,637.35,639.60,630.50,639.25,2385532.0,0.0,1.0,637.350000,639.600000,630.500000,639.250000,2385532.0
2024-01-28,LLY,637.35,639.60,630.50,639.25,2385532.0,0.0,1.0,637.350000,639.600000,630.500000,639.250000,2385532.0
2024-01-29,LLY,641.50,645.65,639.77,645.00,2664450.0,0.0,1.0,641.500000,645.650000,639.770000,645.000000,2664450.0


In [36]:
display(Markdown("""
### To calculate pe now, we take the two dataframes and merge on the index `tradable_date'.
***
"""))

display(Markdown("""
# LLY EPS DATAFRAME
***
"""))
lly_eps = extract_expand_ffill(eps_df.loc[('LLY'),:])
display(lly_eps)

display(Markdown("""
# LLY PRICES DATAFRAME
***
"""))
display(lly_prices[['adj_close']])

display(Markdown("""
# LLY PE (MERGED) DATAFRAME
***
"""))
pe_df = lly_eps.merge(
    lly_prices,
    how='left',
    left_index=True, right_index=True
)
pe_df['pe'] = pe_df['adj_close'] / pe_df['processed_eps']
display(pe_df[['pe', 'processed_eps', 'adj_close']])


### To calculate pe now, we take the two dataframes and merge on the index `tradable_date'.
***



# LLY EPS DATAFRAME
***


,per_end_date,per_type,filing_date,eps_diluted_net,basic_net_eps,processed_eps
2006-05-03,2006-03-31,Q,2006-05-02,0.77,0.77,0.770
2006-05-04,2006-03-31,Q,2006-05-02,0.77,0.77,0.770
2006-05-05,2006-03-31,Q,2006-05-02,0.77,0.77,0.770
2006-05-06,2006-03-31,Q,2006-05-02,0.77,0.77,0.770
2006-05-07,2006-03-31,Q,2006-05-02,0.77,0.77,0.770
...,...,...,...,...,...,...
2023-10-30,2023-06-30,Q,2023-08-08,1.95,1.95,1.950
2023-10-31,2023-06-30,Q,2023-08-08,1.95,1.95,1.950
2023-11-01,2023-06-30,Q,2023-08-08,1.95,1.95,1.950
2023-11-02,2023-06-30,Q,2023-08-08,1.95,1.95,1.950



# LLY PRICES DATAFRAME
***


,adj_close
1972-06-01,1.475751
1972-06-02,1.492844
1972-06-03,1.492844
1972-06-04,1.492844
1972-06-05,1.472788
...,...
2024-01-26,639.250000
2024-01-27,639.250000
2024-01-28,639.250000
2024-01-29,645.000000



# LLY PE (MERGED) DATAFRAME
***


,pe,processed_eps,adj_close
2006-05-03,38.770771,0.770,29.853493
2006-05-04,38.815541,0.770,29.887966
2006-05-05,39.502013,0.770,30.416550
2006-05-06,39.502013,0.770,30.416550
2006-05-07,39.502013,0.770,30.416550
...,...,...,...
2023-10-30,289.571992,1.950,564.665385
2023-10-31,283.542122,1.950,552.907137
2023-11-01,283.813415,1.950,553.436159
2023-11-02,297.035109,1.950,579.218462


# VALIDATION

In [38]:
validation_data = {
    'Debt To Mkt Cap': [2.266511, 1.958932, 1.790877, 1.820469, 2.067135, 2.087139, 1.771630, 1.733494, 1.736698, 1.725479,
                        1.535812, 1.533542, 1.628426, 2.089332, 1.940811, 1.951675, 1.779911, 1.423720, 1.377587, 1.380796,
                        1.526555, 1.677865, 1.600181, 1.566785, 1.407799, 1.450422, 1.221106, 1.241113, 1.097919, 1.504358],
    
    'Return On Inv': [5.972855, 4.313252, 3.943927, 4.008969, 4.550975, 7.970179, 6.767552, 6.622135, 6.634354, 7.645313,
                      6.802995, 6.792915, 7.214236, 4.390834, 4.077122, 4.100062, 3.737540, 5.437435, 5.260454, 5.272760,
                      5.832136, 8.460450, 8.066864, 7.897718, 7.092939, 3.872644, 3.265751, 3.318780, 2.938902, 5.366827],
    
    'Price To Earnings': [159.232948, 202.232157, 221.209527, 217.613731, 191.646411, 126.855597, 149.447249, 152.735015,
                          152.453207, 135.281225, 151.987938, 152.212945, 143.343896, 282.497026, 304.115168, 302.422243,
                          331.606389, 217.724877, 225.016059, 224.493252, 203.058101, 152.048436, 159.429879, 162.828128,
                          181.216741, 263.944889, 313.511980, 308.458138, 348.688320, 268.798647]
}

# Dates as the index
validation_index = pd.to_datetime([
    '2021-10-27', '2021-10-28', '2021-12-31', '2022-01-03', '2022-02-23', '2022-02-24', '2022-03-31', '2022-04-01',
    '2022-04-29', '2022-05-02', '2022-06-30', '2022-07-01', '2022-08-04', '2022-08-05', '2022-09-30', '2022-10-03',
    '2022-11-01', '2022-11-02', '2022-12-30', '2023-01-03', '2023-02-22', '2023-02-23', '2023-03-31', '2023-04-03',
    '2023-04-27', '2023-04-28', '2023-06-30', '2023-07-03', '2023-08-08', '2023-08-09'
])

# Create DataFrame
ratio_validation_df = pd.DataFrame(validation_data, index=validation_index)
ratio_validation_df.index.name = 'trade_date'

# Display the resulting DataFrame
validation_df = ratio_validation_df.merge(
    pe_df[['pe']],
    left_index=True, right_index=True,
    how='left'
)
display(Markdown(f"""
'pe' == 'Price To Earnings' => {np.allclose(validation_df['Price To Earnings'], validation_df['pe'])}
"""))
display(validation_df)


'pe' == 'Price To Earnings' => True


,Debt To Mkt Cap,Return On Inv,Price To Earnings,pe
trade_date,,,,
2021-10-27,2.266511,5.972855,159.232948,159.232948
2021-10-28,1.958932,4.313252,202.232157,202.232157
2021-12-31,1.790877,3.943927,221.209527,221.209527
2022-01-03,1.820469,4.008969,217.613731,217.613731
2022-02-23,2.067135,4.550975,191.646411,191.646411
2022-02-24,2.087139,7.970179,126.855597,126.855597
2022-03-31,1.771630,6.767552,149.447249,149.447249
2022-04-01,1.733494,6.622135,152.735015,152.735015
2022-04-29,1.736698,6.634354,152.453207,152.453207
